In [18]:
import requests
from bs4 import BeautifulSoup
#Getting a feel for the html code of the website, since we are not going in detail with this scraper the body html is very large and is examined in a text editor
url = "https://opencritic.com/browse/all"
response = requests.get(url)
print(response.text)


2023-04-13 14:26:21 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): opencritic.com:443
2023-04-13 14:26:21 [urllib3.connectionpool] DEBUG: https://opencritic.com:443 "GET /browse/all HTTP/1.1" 200 None


<!DOCTYPE html><html lang="en"><head>
  <meta charset="utf-8">
  <title>Best Video Games of This Generation - OpenCritic</title>
  <base href="/">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script>
    if (document && document.cookie && document.cookie.indexOf('oc_remove_ads=1') >= 0) {
      (function () {
        var script = '.oc-ad, .nn-player-floating { display: none!important; }';
        var linkElement = this.document.createElement('link');
        linkElement.setAttribute('rel', 'stylesheet');
        linkElement.setAttribute('type', 'text/css');
        linkElement.setAttribute('href', 'data:text/css;charset=UTF-8,' + encodeURIComponent(script));
        var head = document.getElementsByTagName('head')[0];
        head.appendChild(linkElement);
      })();
    }
  </script>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-SZ64N8QEK5"></script>
  <script>
    window.dataLayer = window.dataLayer

In [1]:
import scrapy

class OpenCriticSpider2(scrapy.Spider):
    name = 'opencritic2'
    allowed_domains = ['opencritic.com']
    start_urls = ['https://opencritic.com/browse/all']

    def parse(self, response):
        #scraping game items
        for game in response.css('div.game-row'):
            #Getting both css and xpath of the variables
            rank_css = game.css('div.rank div::text').get()
            rank_xpath = game.xpath("normalize-space(div[@class='rank']/text())").get()
            #Also normalizing space for the xpath
            score_css = game.css('div.score::text').get()
            score_xpath = game.xpath("normalize-space(div[@class='score']/text())").get()
            
            tier_css = game.css('div.tier app-tier-display img::attr(alt)').get()
            tier_xpath = game.xpath("normalize-space(div[@class='tier']/app-tier-display/img/@alt)").get()
            
            name_css = game.css('div.game-name a::text').get()
            name_xpath = game.xpath("normalize-space(div[@class='game-name']/a/text())").get()
            
            platforms_css = game.css('div.platforms::text').get()
            platforms_xpath = game.xpath("normalize-space(div[@class='platforms']/text())").get()
            
            release_date_css = game.css('div.release-date span::text').get()
            release_date_xpath = game.xpath("normalize-space(div[@class='release-date']/span/text())").get()
            #Getting css for everything
            yield {
                'rank': rank_css,
                'score': score_css,
                'tier': tier_css,
                'name': name_css,
                'platforms': platforms_css,
                'release_date': release_date_css
            }

        #This code goes to the next page of the website
        next_page = response.css('div.col-md-12 a[rel="next"]::attr(href)').get()
        if next_page:
            current_page = int(next_page.split('=')[-1])
            #We added a max of 200 to decrease time spend scraping since we have a larger metacritic one already
            if current_page <= 200:
                yield scrapy.Request(response.urljoin(next_page), callback=self.parse)



In [2]:
import os
from scrapy.crawler import CrawlerProcess

#setting the output file path and name
output_file = os.path.join(os.path.expanduser("~"), "OneDrive", "Documenten", "Master", "Online data mining", "Opencritic_output.json")
#Using the settings of the base scrapy project
process = CrawlerProcess(settings={
    "FEED_URI": output_file,
    "FEED_FORMAT": "json",
})

process.crawl(OpenCriticSpider2)
process.start()


2023-04-13 14:42:35 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2023-04-13 14:42:35 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.18363-SP0
2023-04-13 14:42:35 [scrapy.crawler] INFO: Overridden settings:
{}
2023-04-13 14:42:35 [py.warnings] WARNING: d:\masterprograms\Anaconda\envs\MISY350\lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation 

In [3]:
import json

with open('C:/Users/Ward/Documents/Scrapy/opencritic_scraper2/opencritic_scraper2/spiders/output.json', 'r') as f:
    data = json.load(f)

cleaned_data = []

for item in data:
    # Remove leading/trailing whitespaces from string values
    cleaned_item = {key: val.strip() if isinstance(val, str) else val for key, val in item.items()}
    cleaned_data.append(cleaned_item)

headers = ['rank', 'score', 'tier', 'name', 'platforms', 'release_date']
combined_data = {header: [] for header in headers}

for item in cleaned_data:
    for header in headers:
        combined_data[header].append(item[header])

print(combined_data)


{'rank': ['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '11.', '12.', '13.', '14.', '15.', '16.', '17.', '18.', '19.', '20.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '21.', '22.', '23.', '24.', '25.', '26.', '27.', '28.', '29.', '30.', '31.', '32.', '33.', '34.', '35.', '36.', '37.', '38.', '39.', '40.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '41.', '42.', '43.', '44.', '45.', '46.', '47.', '48.', '49.', '50.', '51.', '52.', '53.', '54.', '55.', '56.', '57.', '58.', '59.', '60.', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '61.', '62.', '63.', '64.', '65.', '66.', '67.', '68.', '69.', '70.', '71.'

In [4]:
import pandas as pd
import json
#Importing the json output file
with open('C:/Users/Ward/Documents/Scrapy/opencritic_scraper2/opencritic_scraper2/spiders/output.json', 'r') as f:
    data = json.load(f)

cleaned_data = []

for item in data:
    #Removing leading/trailing whitespaces from string values
    cleaned_item = {key: val.strip() if isinstance(val, str) else val for key, val in item.items()}
    cleaned_data.append(cleaned_item)

df = pd.DataFrame(cleaned_data, columns=['rank', 'score', 'tier', 'name', 'platforms', 'release_date'])
#Dropping any columns that do not have any data
df = df.dropna()

print(df.head(100))



2023-04-13 14:47:27 [numexpr.utils] INFO: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-04-13 14:47:27 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


     rank score    tier                                     name  \
0      1.    97  Mighty                      Super Mario Odyssey   
1      2.    96  Mighty  The Legend of Zelda: Breath of the Wild   
2      3.    96  Mighty                    Red Dead Redemption 2   
3      4.    95  Mighty                               Elden Ring   
4      5.    94  Mighty                               God of War   
..    ...   ...     ...                                      ...   
207   96.    89  Mighty                           Rogue Legacy 2   
208   97.    89  Mighty             Tony Hawk's Pro Skater 1 + 2   
209   98.    89  Mighty                            Shovel Knight   
210   99.    89  Mighty  Xenoblade Chronicles Definitive Edition   
211  100.    89  Mighty                  Mario Kart 8 DLC Pack 2   

                               platforms  release_date  
0                                 Switch  Oct 27, 2017  
1                          Wii-U, Switch   Mar 3, 2017  
2        PS4

In [5]:
import pandas as pd

#Printing the amount of rows after deleting the nan's 
num_rows = df.shape[0]
print(f"The number of rows in my dataset is {num_rows}.")


The number of rows in my dataset is 1000.
